In [ ]:
import os
import tarfile
import random
import tensorflow as tf
import cv2
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

In [ ]:
!wget --no-check-certificate \
http://leafsnap.com/static/dataset/leafsnap-dataset.tar \
    -O /tmp/dataset.tar

--2020-05-21 20:51:21--  http://leafsnap.com/static/dataset/leafsnap-dataset.tar
Resolving leafsnap.com (leafsnap.com)... 128.59.23.133
Connecting to leafsnap.com (leafsnap.com)|128.59.23.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1023416320 (976M) [application/x-tar]
Saving to: ‘/tmp/dataset.tar’

/tmp/dataset.tar    100%[===================>] 976.01M  89.0MB/s    in 14s     

2020-05-21 20:51:35 (70.2 MB/s) - ‘/tmp/dataset.tar’ saved [1023416320/1023416320]



In [ ]:
local_zip = '/tmp/dataset.tar'
zip_ref   = tarfile.open(local_zip, 'r')
zip_ref.extractall('/content/tmp')
zip_ref.close()


In [ ]:
data_dir="/content/tmp/dataset/images/field"
train_dir='/content/tmp/train/'
test_dir='/content/tmp/test/'
data_lis = os.listdir(data_dir)
print(data_lis)
try:
  if(not os.path.exists('/content/tmp/train')):
    os.mkdir('/content/tmp/'+'train')
  if(not os.path.exists('/content/tmp/test')):
    os.mkdir('/content/tmp/'+'test')
  for leaf_name in data_lis:
    if(not os.path.exists('/content/tmp/train/'+leaf_name)):
      os.mkdir('/content/tmp/train/'+leaf_name)
    if(not os.path.exists('/content/tmp/test/'+leaf_name)):
      os.mkdir('/content/tmp/test/'+leaf_name)
except:
  pass




['syringa_reticulata', 'ostrya_virginiana', 'malus_baccata', 'catalpa_bignonioides', 'crataegus_phaenopyrum', 'morus_alba', 'quercus_phellos', 'styrax_obassia', 'pinus_sylvestris', 'corylus_colurna', 'ulmus_glabra', 'magnolia_denudata', 'populus_deltoides', 'quercus_montana', 'staphylea_trifolia', 'ulmus_americana', 'crataegus_pruinosa', 'malus_hupehensis', 'asimina_triloba', 'gleditsia_triacanthos', 'ailanthus_altissima', 'cercis_canadensis', 'prunus_serrulata', 'abies_concolor', 'ulmus_parvifolia', 'acer_saccharum', 'ficus_carica', 'crataegus_crus-galli', 'chionanthus_retusus', 'acer_pseudoplatanus', 'pinus_wallichiana', 'pinus_thunbergii', 'quercus_shumardii', 'aesculus_pavi', 'castanea_dentata', 'amelanchier_laevis', 'abies_nordmanniana', 'chamaecyparis_pisifera', 'pseudolarix_amabilis', 'catalpa_speciosa', 'taxodium_distichum', 'acer_griseum', 'fraxinus_americana', 'tilia_cordata', 'eucommia_ulmoides', 'populus_tremuloides', 'gymnocladus_dioicus', 'quercus_coccinea', 'zelkova_serr

In [ ]:
import glob
for fold in data_lis:
  fn=glob.glob('/content/tmp/test/'+fold+'/*.jpg') #gives a list of files
  for i in fn:
      if os.path.exists(i):
        os.remove(i)

In [ ]:
import numpy as np
def split_data(SOURCE,train_dir,test_dir,split_size):
  files = []
  for filename in os.listdir(SOURCE):
    file = SOURCE + filename
    if os.path.getsize(file) > 0:
      files.append(filename)
    else:
      print(filename + " is zero length, so ignoring.")
  training_length = int(len(files) * split_size)
  testing_length = int(len(files) - training_length)
  shuffled_set = random.sample(files, len(files))
  training_set = shuffled_set[0:training_length]
  testing_set = shuffled_set[-testing_length:]
  h1=224
  w1=224
  dimension=(w1,h1)
  for filename in training_set:
    this_file = str(SOURCE + filename)
    destination = str(train_dir + filename)
    img=cv2.imread(this_file)
    #kernel = np.ones((5,5),np.float32)/25 #4x4 window size and the valuse with that windowa are summed up and diveide by(16) which is 4*4. 
    #dst = cv2.filter2D(this_file,-1,kernel)
    re_img=cv2.resize(img,dimension,interpolation=cv2.INTER_AREA)
    cv2.imwrite(this_file,re_img)
    copyfile(this_file, destination)
  for filename in testing_set:
    this_file =str(SOURCE+filename)
    destination = str(test_dir+filename)
    img=cv2.imread(this_file)
    #kernel = np.ones((5,5),np.float32)/25 #4x4 window size and the valuse with that windowa are summed up and diveide by(16) which is 4*4. 
    #dst = cv2.filter2D(this_file,-1,kernel)
    re_img=cv2.resize(img,dimension,interpolation=cv2.INTER_AREA)
    cv2.imwrite(this_file,re_img)
    copyfile(this_file,destination)
#for each file in RGB directory
source_dir="/content/tmp/dataset/images/field/"
train_dir='/content/tmp/train/'
test_dir='/content/tmp/test/'
data_lis = os.listdir(data_dir)
for dir_name in data_lis:
  SOURCE = source_dir+dir_name+"/"
  TRAINING = train_dir+dir_name+"/"
  TESTING = test_dir+dir_name+"/"
  split_data(SOURCE,TRAINING,TESTING,0.99)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.999):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(
    rescale=1. / 255)


In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, Input
from keras.models import Model
model = VGG16(input_tensor=Input(shape=(224, 224, 3)),weights='imagenet', include_top=False)


x=Flatten()(model.output)
x = Dense(4096,activation = 'relu')(x)
x=Dense(184, activation='softmax')(x)
model=Model(model.input,x)
#for layer in model.layers:
  #layer.trainable = False
model.summary()
print(len(model.layers))

Using TensorFlow backend.


58892288/58889256 [==============================] - 1s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [ ]:

'''model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(50, 50, 3)),

    tf.keras.layers.MaxPooling2D(2, 2),
  
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(40, activation='softmax')
])'''
import keras
model.compile(optimizer = keras.optimizers.Adam(),
              loss = 'categorical_crossentropy',
              metrics=['acc'])

In [ ]:
TRAINING_DIR = "/content/tmp/train/"
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(224, 224))
VALIDATION_DIR = "/content/tmp/test/"
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=32,
                                                              class_mode='categorical',
                                                              target_size=(224, 224))


Found 7528 images belonging to 184 classes.
Found 191 images belonging to 184 classes.


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.axis([0,25, 0, 1])
plt.figure()
plt.plot(epochs, loss, 'r', label='Training accuracy')
plt.plot(epochs, val_loss, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.axis([0,25, 0, 1])
plt.figure()

plt.show()

NameError: ignored

In [ ]:
#model.summary()
#model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit_generator(train_generator, epochs=50 ,validation_data = validation_generator)

model.save("leafdata.h5")

Epoch 1/50
236/236 [==============================] - 84s 354ms/step - loss: 5.3467 - acc: 0.0221 - val_loss: 5.3078 - val_acc: 0.0105
Epoch 2/50
236/236 [==============================] - 76s 322ms/step - loss: 5.0817 - acc: 0.0221 - val_loss: 5.1063 - val_acc: 0.0105
Epoch 3/50
236/236 [==============================] - 77s 327ms/step - loss: 5.0751 - acc: 0.0240 - val_loss: 5.1642 - val_acc: 0.0105
Epoch 4/50
236/236 [==============================] - 77s 327ms/step - loss: 5.0748 - acc: 0.0228 - val_loss: 5.2210 - val_acc: 0.0105
Epoch 5/50
236/236 [==============================] - 77s 325ms/step - loss: 5.0716 - acc: 0.0239 - val_loss: 5.3056 - val_acc: 0.0105
Epoch 6/50
236/236 [==============================] - 77s 326ms/step - loss: 5.0716 - acc: 0.0236 - val_loss: 5.3177 - val_acc: 0.0105
Epoch 7/50
236/236 [==============================] - 77s 326ms/step - loss: 5.0711 - acc: 0.0213 - val_loss: 5.1970 - val_acc: 0.0105
Epoch 8/50
236/236 [==============================] - 7

In [ ]:
!

In [ ]:
!git init

Initialized empty Git repository in /content/.git/


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
#model=tf.keras.models.load_model('/content/rps.h5')
path = '/content/tmp/train/liriodendron_tulipifera/12992002367158.jpg'
img = image.load_img(path, target_size=(224, 224))
y = tf.random.normal(shape=(224, 224, 3))
x=tf.image.central_crop(y, 0.5)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images, batch_size=10)
data_lis.sort()
max_val=max(classes)
print(classes)
print(len(data_lis))
data_lis.sort()
print(data_lis[np.argmax(classes[0])])

In [ ]:
!git commit -m "first commit"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@59ee12f391ed.(none)')
